

# Latihan Preprocessing Data untuk Data Mining

**Nama:** Deni anjas rori rilawan
**NIM:** A11.2023.15153
**Dataset:** Indeks Pembangunan Manusia (IPM) per Provinsi
**Sumber:** Badan Pusat Statistik (BPS), data tahun 2023
```


In [1]:
# Import library yang diperlukan
import pandas as pd
import numpy as np
import io # Digunakan untuk membaca string sebagai file
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

print("Library berhasil di-import.")

Library berhasil di-import.


-----

#### **Langkah 2: Memuat Dataset**

Untuk memudahkan, saya sudah siapkan datanya dalam format CSV di dalam kode. Data ini diambil dari tabel resmi BPS untuk IPM tahun 2023. Saya sengaja menambahkan beberapa `np.nan` (missing values) dan baris duplikat untuk simulasi proses pembersihan data.

In [2]:
# Data IPM 2023 dari BPS dalam format string CSV
# Sumber: https://www.bps.go.id/indicator/26/49/1/indeks-pembangunan-manusia.html
# Beberapa nilai sengaja dikosongkan (NaN) dan diduplikasi untuk latihan

csv_data = """Provinsi,UHH,HLS,RLS,Pengeluaran,IPM
ACEH,70.6,13.98,9.45,10313,74.7
SUMATERA UTARA,70.52,13.79,9.91,11443,75.13
SUMATERA BARAT,70.71,14.1,9.98,11894,75.64
RIAU,71.55,13.89,10.02,13155,76.99
JAMBI,71.69,13.56,9.42,11849,74.07
SUMATERA SELATAN,70.51,13.3,8.78,12173,73.18
BENGKULU,70.73,13.82,9.37,11211,73.49
LAMPUNG,71.18,13.34,8.81,10886,72.48
KEP. BANGKA BELITUNG,71.7,12.98,9.08,13996,74.77
KEP. RIAU,73.18,13.76,10.51,14470,78.25
DKI JAKARTA,73.66,13.1,11.42,20016,83.55
JAWA BARAT,71.17,12.9,8.89,12185,74.25
JAWA TENGAH,72.68,12.8,8.23,11511,73.39
DI YOGYAKARTA,75.69,15.65,9.85,14271,81.33
JAWA TIMUR,72.3,13.39,8.55,12349,74.65
BANTEN,70.83,12.9,9.58,13019,74.22
BALI,73.48,13.89,9.95,14234,77.46
NUSA TENGGARA BARAT,,13.04,7.85,10332,71.2
NUSA TENGGARA BARAT,,13.04,7.85,10332,71.2
NUSA TENGGARA TIMUR,68.42,13.62,8.44,,68.08
KALIMANTAN BARAT,68.79,12.87,8.24,10165,69.94
KALIMANTAN TENGAH,71.9,13.29,8.96,11844,73.91
KALIMANTAN SELATAN,71.84,13.23,9.15,12112,73.57
KALIMANTAN TIMUR,72.32,13.9,10.22,13778,78.2
KALIMANTAN UTARA,71.49,13.66,9.75,12502,75.19
SULAWESI UTARA,71.59,13.59,9.92,11246,74.5
SULAWESI TENGAH,70.15,13.28,8.95,10515,72.01
SULAWESI SELATAN,70.36,13.61,8.88,11762,73.81
SULAWESI TENGGARA,70.55,13.75,9.25,10931,73.2
GORONTALO,68.32,13.57,9.19,9576,71.18
SULAWESI BARAT,67.8,13.14,8.5,10372,68.93
MALUKU,68.34,13.9,9.6,9082,71.42
MALUKU UTARA,67.64,13.18,8.83,9232,69.97
PAPUA BARAT,64.38,12.99,7.99,8424,67.16
PAPUA,61.76,11.28,7.18,8379,62.99
"""

# Membaca data string menjadi DataFrame pandas
df = pd.read_csv(io.StringIO(csv_data))

# Menampilkan 5 baris pertama data
print("Data Awal (5 Baris Pertama):")
df.head()

Data Awal (5 Baris Pertama):


,Provinsi,UHH,HLS,RLS,Pengeluaran,IPM
0,ACEH,70.60,13.98,9.45,10313.0,74.70
1,SUMATERA UTARA,70.52,13.79,9.91,11443.0,75.13
2,SUMATERA BARAT,70.71,14.10,9.98,11894.0,75.64
3,RIAU,71.55,13.89,10.02,13155.0,76.99
4,JAMBI,71.69,13.56,9.42,11849.0,74.07


-----

#### **Langkah 3: Inspeksi Data Awal**

Kita periksa kondisi data, seperti tipe data, jumlah data yang hilang, dan data duplikat.

In [3]:
# Menampilkan informasi ringkas tentang DataFrame
print("Informasi DataFrame:")
df.info()

print("\n==================================================\n")

# Mengecek jumlah missing values
print("Jumlah Missing Values Sebelum Dibersihkan:")
print(df.isnull().sum())

print("\n==================================================\n")

# Mengecek jumlah data duplikat
print(f"Jumlah baris duplikat sebelum dihapus: {df.duplicated().sum()}")

Informasi DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Provinsi     35 non-null     object 
 1   UHH          33 non-null     float64
 2   HLS          35 non-null     float64
 3   RLS          35 non-null     float64
 4   Pengeluaran  34 non-null     float64
 5   IPM          35 non-null     float64
dtypes: float64(5), object(1)
memory usage: 1.8+ KB


Jumlah Missing Values Sebelum Dibersihkan:
Provinsi       0
UHH            2
HLS            0
RLS            0
Pengeluaran    1
IPM            0
dtype: int64


Jumlah baris duplikat sebelum dihapus: 1


-----

#### **Langkah 4: Membersihkan Data (Data Cleaning)**

Berdasarkan inspeksi, kita perlu menangani *missing values* dan data duplikat.

##### **4.1. Menangani Data Duplikat**

Langkah pertama dan paling mudah adalah menghapus baris data yang sama persis.

In [4]:
# Menghapus baris duplikat
df.drop_duplicates(inplace=True)
print(f"Jumlah baris duplikat setelah dihapus: {df.duplicated().sum()}")
print(f"Ukuran data setelah menghapus duplikat: {df.shape}")

Jumlah baris duplikat setelah dihapus: 0
Ukuran data setelah menghapus duplikat: (34, 6)


##### **4.2. Menangani Missing Values (Imputasi)**

Kita akan mengisi nilai yang kosong. Karena semua datanya numerik, kita bisa menggunakan **mean** (rata-rata) atau **median** (nilai tengah). Kita akan menggunakan **mean**.

In [5]:
# Inisialisasi imputer dengan strategi 'mean'
imputer = SimpleImputer(strategy='mean')

# Kita hanya akan melakukan imputasi pada kolom numerik
numeric_cols = df.select_dtypes(include=np.number).columns

# Terapkan imputer ke DataFrame
# Note: imputer.fit_transform menghasilkan numpy array, jadi kita buat kembali ke DataFrame
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Verifikasi kembali jumlah missing values
print("\nJumlah Missing Values setelah imputasi:")
print(df.isnull().sum())


Jumlah Missing Values setelah imputasi:
Provinsi       0
UHH            0
HLS            0
RLS            0
Pengeluaran    0
IPM            0
dtype: int64


-----

#### **Langkah 5: Transformasi Data (Data Transformation)**

Komponen-komponen IPM (`UHH`, `HLS`, `RLS`, `Pengeluaran`) memiliki rentang nilai yang berbeda. Misalnya, `UHH` berkisar 60-75, sedangkan `Pengeluaran` mencapai puluhan ribu. Kita perlu menyamakan skalanya agar seimbang. Proses ini disebut **Normalisasi**. Kita akan menggunakan `MinMaxScaler` yang akan mengubah skala data ke rentang [0, 1].

In [6]:
# Kolom 'Provinsi' adalah identifier, bukan fitur numerik, jadi kita kecualikan dari scaling
# Kita buat DataFrame baru khusus untuk data yang akan di-scaling
data_to_scale = df[numeric_cols]

# Inisialisasi MinMaxScaler
scaler = MinMaxScaler()

# Lakukan scaling pada data
scaled_data = scaler.fit_transform(data_to_scale)

# Konversi hasil scaling (numpy array) kembali menjadi DataFrame
df_scaled = pd.DataFrame(scaled_data, columns=numeric_cols)

# Gabungkan kembali dengan kolom 'Provinsi' untuk melihat hasil akhir yang rapi
df_final = pd.concat([df['Provinsi'].reset_index(drop=True), df_scaled], axis=1)


print("Data Setelah Normalisasi (5 Baris Pertama):")
df_final.head()

Data Setelah Normalisasi (5 Baris Pertama):


,Provinsi,UHH,HLS,RLS,Pengeluaran,IPM
0,ACEH,0.634602,0.617849,0.535377,0.166194,0.569553
1,SUMATERA UTARA,0.628859,0.574371,0.643868,0.263298,0.590467
2,SUMATERA BARAT,0.642498,0.645309,0.660377,0.302054,0.615272
3,RIAU,0.702800,0.597254,0.669811,0.410415,0.680934
4,JAMBI,0.712850,0.521739,0.528302,0.298187,0.538911


**Penjelasan:** Seperti yang terlihat, semua nilai numerik sekarang berada dalam rentang antara 0 dan 1. Data ini jauh lebih baik dan siap untuk diproses oleh algoritma machine learning.

-----

### **Cara Mengunggah ke GitHub**

Langkah-langkahnya sama seperti sebelumnya:

1.  **Simpan File:** Pastikan Anda menyimpan notebook ini dengan nama `preprocessing.ipynb`.
2.  **Buat Repositori Baru di GitHub:** Buat repositori publik baru (misal: `Tugas-DataMining-BPS`).
3.  **Unggah File:** Di dalam repositori, pilih **Add file** \> **Upload files**, lalu unggah file `preprocessing.ipynb` Anda.
4.  **Salin dan Kirim URL:** Salin URL repositori Anda (contoh: `https://github.com/nama-anda/Tugas-DataMining-BPS`) dan kirimkan melalui Kulino.

Semoga ini sesuai dengan yang Anda harapkan\! 😊

Untuk memberikan gambaran umum tentang bagaimana dataset publik bisa ditemukan dan digunakan, video ini menjelaskan berbagai sumber dataset publik, termasuk dari instansi pemerintah seperti BPS.